In [0]:
%sql
CREATE CATALOG IF NOT EXISTS finance_catalog
MANAGED LOCATION 'abfss://synthetic-financial-dataset@incrementalingestion.dfs.core.windows.net/catalog';


In [0]:
%sql
USE CATALOG finance_catalog;

In [0]:
%sql
SHOW SCHEMAS IN finance_catalog;

databaseName
bronze
default
gold
information_schema
silver


In [0]:
%sql
DESCRIBE CATALOG finance_catalog;

info_name,info_value
Catalog Name,finance_catalog
Comment,
Owner,pateljeet4999@gmail.com
Catalog Type,Regular


In [0]:
%sql 
CREATE SCHEMA IF NOT EXISTS finance_catalog.bronze;
CREATE SCHEMA IF NOT EXISTS finance_catalog.silver;
CREATE SCHEMA IF NOT EXISTS finance_catalog.gold;


In [0]:
%sql
--Create Bronze table ONCE 
CREATE TABLE IF NOT EXISTS finance_catalog.bronze.transactions
USING DELTA 
AS 
SELECT *
FROM csv.`abfss://synthetic-financial-dataset@incrementalingestion.dfs.core.windows.net/catalog/landing`
WITH (
  header = "true",
  inferSchema = "true"
); 

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Step 1 after new files lands in landing zone
COPY INTO finance_catalog.bronze.transactions
FROM 'abfss://synthetic-financial-dataset@incrementalingestion.dfs.core.windows.net/catalog/landing'
FILEFORMAT = CSV
FORMAT_OPTIONS (
  'header' = 'true',
  'inferSchema' = 'true'
)
COPY_OPTIONS (
  'mergeSchema' = 'true'
); 

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
6668,6668,0


In [0]:
%sql
-- Verify new files have updated into the database
SELECT COUNT(*) FROM finance_catalog.bronze.transactions;


COUNT(*)
10001


In [0]:
%sql
DROP TABLE IF EXISTS finance_catalog.silver.transactions;
DROP TABLE IF EXISTS finance_catalog.bronze.transactions;

-- Drop watermark table if exists
DROP TABLE IF EXISTS finance_catalog.silver.watermark;